In [17]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # set to n-1 to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation #toggle this on when MSB set to 0
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc

In [20]:
""" Function """
def ctrl_shift(qc, a, ctrl_value, c, direction):
    """
    This function shifts input 'a' left or right by 1 qubit. Overflow/underflow information is lost.
    ### THIS DOES NOT HAVE OVERFLOW OR UNDERFLOW HANDLING ###

    Parameters:
    - qc: QuantumCircuit on which the shift is performed
    - a: QuantumRegister for the number we wish to shift
    - ctrl_value: This qubit determines whether a shift is performed
    - c: Ancillaries qubits for temporary storage
    - direction: The direction of the shift; 'r' or 'R' for right, 'l' or 'L' for left

    Returns: 
    - qc: The updated quantum circuit
    """
    
    n = len(a)  # Number of bits in the register

    """ Direction handling """
    if direction.lower() == "l":  # Left shift

        # Handle the overflow by moving the most significant bit to the ancillary qubit
        qc.cx(a[n-1], c[0])
        qc.ccx(ctrl_value, c[0], a[n-1])
        qc.cx(a[n-1], c[0])
        
        # Shift the qubits to the left
        for i in range(n-1, 0, -1):
            qc.cx(a[i-1], a[i])
            qc.ccx(ctrl_value, a[i], a[i-1])
            qc.cx(a[i-1], a[i])
        
        
        # Reset the ancillary qubit
        qc.reset(c[0])
        
    elif direction.lower() == "r":  # Right shift
        # Handle the underflow by moving the least significant bit to the ancillary qubit
        qc.cx(a[0], c[0])
        qc.ccx(ctrl_value, c[0], a[0])
        qc.cx(a[0], c[0])
        
        # Shift the qubits to the right
        for i in range(1, n):
            qc.cx(a[i], a[i-1])
            qc.ccx(ctrl_value, a[i-1], a[i],)
            qc.cx(a[i], a[i-1])
        
        # Reset the ancillary qubit
        qc.reset(c[0])

    else:
        raise ValueError("Invalid direction input. Use 'l'/'L' for left or 'r'/'R' for right.")
        
    return qc


In [28]:
""" MAIN, This does a x b"""

# Number of bits
n = 5

ancillaries = QuantumRegister(n, 'ancillaries')
ctrl = QuantumRegister(1, 'ctrl')
a = QuantumRegister(n, 'a')
#c_out = QuantumRegister(1, 'c_out')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister( n + 2*n, 'result')

""" Repeated a few times for checking """
for k in range(0,7):
    ancillaries = QuantumRegister(n, 'ancillaries')
    ctrl = QuantumRegister(1, 'ctrl')
    a = QuantumRegister(n, 'a')
    #c_out = QuantumRegister(1, 'c_out')
    
    # Define a classical register with 2n bits (n for a and n for b)
    result = ClassicalRegister( n + 2*n, 'result')

    qc = QuantumCircuit(a, ancillaries, result, ctrl)
    
    """initialize the quantum numbers"""
    qc = init_random_num(qc, a)
    qc.x(ctrl[0])
    
    #for i in range (0, n):
    #    qc.x(a[i])
    
    """apply the ctrl shift operation"""
    qc = ctrl_shift(qc, a, ctrl[0], ancillaries, "r")
    
    
    # Measure the qubits into the classical register
    qc.measure(a, result[:n])      # Measure 'a' into the first n classical bits
    qc.measure(ancillaries, result[n:len(ancillaries)+n]) # Measure the ancilla's just to check they are still 0
    
    
    # For execution
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    sim_result = simulator.run(compiled_circuit).result()
    counts = sim_result.get_counts()
    
    #print(counts)
    
    print("Measurement results:")
    for bitstring, count in counts.items():
        # bitstring is of the form 'result' where the first n bits are for 'a' and the last n bits are for 'b'
        a_result = bitstring[-n:]  # Last n bits for b
        ancillaries = bitstring[:-n]  # First 2n bits for a x b
        a_decimal_value = int(a_result, 2)
    
        print(f"After shift: a = {a_result} (decimal: {a_decimal_value}), ancillaries = {ancillaries}\n")
    
    del qc # delete circuit for next iteration


a: 00001 (decimal: 1)
Measurement results:
After shift: a = 00000 (decimal: 0), ancillaries = 0000000000

a: 01010 (decimal: 10)
Measurement results:
After shift: a = 00101 (decimal: 5), ancillaries = 0000000000

a: 01010 (decimal: 10)
Measurement results:
After shift: a = 00101 (decimal: 5), ancillaries = 0000000000

a: 00110 (decimal: 6)
Measurement results:
After shift: a = 00011 (decimal: 3), ancillaries = 0000000000

a: 00101 (decimal: 5)
Measurement results:
After shift: a = 00010 (decimal: 2), ancillaries = 0000000000

a: 00110 (decimal: 6)
Measurement results:
After shift: a = 00011 (decimal: 3), ancillaries = 0000000000

a: 00000 (decimal: 0)
Measurement results:
After shift: a = 00000 (decimal: 0), ancillaries = 0000000000



In [25]:
list=[]
mult_reg=[]
list_a = ['a','b','c','d']
num_add = len(list_a)

for w in range(0,10):
    list.append(w)

print(list[1:])
print(list[-4:])

for w in range(0,19):
    mult_reg.append(-w)

print(list[-4:-1])
n=len(list)
for i in range(1,num_add-1):
    print(str(list))
    print("\n")
    print(mult_reg[i:i+n])
    print("\n")
    print(list_a[i]+"/n")

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[6, 7, 8, 9]
[6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10]


b/n
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[-2, -3, -4, -5, -6, -7, -8, -9, -10, -11]


c/n
